In [1]:
# load input data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print('Length of dataset in characters: ', len(text))

Length of dataset in characters:  400354


In [4]:
# get all characters in dataset and sort them
chars = sorted(list(set(text)))
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
# get vocabularly size
vocab_size = len(chars)
print(vocab_size)

65


In [6]:
# build encoder and decoder on character level
stoi = {ch:i for i,ch in enumerate(chars)} # string to integer
itos = {i:ch for i, ch in enumerate(chars)} # integer to string

encode = lambda s: [stoi[c] for c in s] # encoder: takes a string and outputs list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of integers and outputs a string

In [7]:
print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [10]:
# comment, tiktoken tokenizer can be used
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("hello world"))
print(enc.decode(enc.encode("hello world")))

[31373, 995]
hello world


In [12]:
# encode entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:200])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [13]:
# split data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [14]:
# sample chunk of data
block_size = 8
train_data[:block_size+1] # so we have 8 individual examples in respect to context

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t] # as y starts from 1
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [18]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel to train transformer
block_size = 8 # maximum context length for predictions

In [26]:
data = train_data
print(len(data) - block_size)
ix = torch.randint(len(data) - block_size, (batch_size,)) # batch_size = 4
print(ix)

1003846
tensor([975771, 700211, 488136, 649358])


In [28]:
def get_batch(split: str):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [29]:
xb, yb = get_batch('train')
print('inputs: ', xb.shape)
print(xb)
print('targets', yb.shape)
print(yb)

inputs:  torch.Size([4, 8])
tensor([[52, 41, 43,  1, 53, 44,  1, 39],
        [49, 52, 53, 41, 49, 57,  8,  0],
        [56, 53, 52, 45,  1, 39,  1, 44],
        [ 8,  0,  0, 18, 47, 56, 57, 58]])
targets torch.Size([4, 8])
tensor([[41, 43,  1, 53, 44,  1, 39,  1],
        [52, 53, 41, 49, 57,  8,  0, 35],
        [53, 52, 45,  1, 39,  1, 44, 47],
        [ 0,  0, 18, 47, 56, 57, 58,  1]])


In [30]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b,:t+1]
        target = yb[b,t] # as y starts from 1
        print(f"when input is {context} the target: {target}")

when input is tensor([52]) the target: 41
when input is tensor([52, 41]) the target: 43
when input is tensor([52, 41, 43]) the target: 1
when input is tensor([52, 41, 43,  1]) the target: 53
when input is tensor([52, 41, 43,  1, 53]) the target: 44
when input is tensor([52, 41, 43,  1, 53, 44]) the target: 1
when input is tensor([52, 41, 43,  1, 53, 44,  1]) the target: 39
when input is tensor([52, 41, 43,  1, 53, 44,  1, 39]) the target: 1
when input is tensor([49]) the target: 52
when input is tensor([49, 52]) the target: 53
when input is tensor([49, 52, 53]) the target: 41
when input is tensor([49, 52, 53, 41]) the target: 49
when input is tensor([49, 52, 53, 41, 49]) the target: 57
when input is tensor([49, 52, 53, 41, 49, 57]) the target: 8
when input is tensor([49, 52, 53, 41, 49, 57,  8]) the target: 0
when input is tensor([49, 52, 53, 41, 49, 57,  8,  0]) the target: 35
when input is tensor([56]) the target: 53
when input is tensor([56, 53]) the target: 52
when input is tensor(

In [43]:
# simplest baeline: bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
from numpy import log as ln
torch.manual_seed(1337)

In [44]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # C - number of channels

    def forward(self, idx, targets):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)

        B, T, C = logits.shape
        logits = logits.view(B*T, C) # one dimensional array to stretch an arrayy
        targets = targets.view(B*T) # or can be set to -1
        # negative logloss
        loss = F.cross_entropy(logits, targets)

        return logits, loss


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(f"our expected negative logloss: {-ln(1/65)}")

torch.Size([32, 65])
tensor(4.4564, grad_fn=<NllLossBackward0>)
our expected negative logloss: 4.174387269895637


In [46]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # C - number of channels

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # one dimensional array to stretch an arrayy
            targets = targets.view(B*T) # or can be set to -1
            # negative logloss
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # goes to forward
            # take the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(f"our expected negative logloss: {-ln(1/65)}")

torch.Size([32, 65])
tensor(4.4976, grad_fn=<NllLossBackward0>)
our expected negative logloss: 4.174387269895637


In [57]:
# kick off generation
idx = torch.zeros((1,1), dtype=torch.long)

In [59]:
print(decode(m.generate(idx=idx, max_new_tokens=10)[0].tolist()))


PQpbpb
r3N


## train part

In [60]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [68]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    # zero gradients from previous step
    optimizer.zero_grad(set_to_none=True)
    # get gradients for all parameters
    loss.backward()
    # use gradients to update parameters
    optimizer.step()

print(loss.item())

2.289489507675171


In [69]:
print(decode(m.generate(idx=idx, max_new_tokens=200)[0].tolist()))



Whis Fordieng rs man IOWhe's a wn.

ARERAnde lale se allir sie, ththant hashar, ousthaw ady w, atasmppond:
POS:

Ang we thaphen,

Dithechalicof kinoff? her t t'd ure, pe itithecurerang,
TES:

TRCHAn 


## Self-attention

In [70]:
# the mathemathical trick in self-attention

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels

x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [72]:
# we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bag of words

In [73]:
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0) # averaging over time

In [78]:
x[0] # zeros batch element

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [80]:
(0.1808 + -0.3596)/2

-0.0894

In [79]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [81]:
# toy example for usage of matrix multiplication
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)
print('---')

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
---


In [83]:
# low triangular tensor
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [84]:
# toy example for usage of matrix multiplication
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)
print('---')

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
---


In [85]:
# normalize over rows
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)
print('---')


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---


In [89]:
# use the trick
wei = torch.tril(torch.ones(T,T)) # array of weights
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [90]:
xbow2 = wei @ x # (_B_, T, T) @ (B, T, C) ----> (B, T, C)

In [91]:
torch.allclose(xbow, xbow2)

True

In [92]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [94]:
trill = torch.tril(torch.ones(T,T)) 
trill

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [104]:
wei = torch.zeros((T,T)) # affinities between tokens
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [105]:
wei = wei.masked_fill(trill == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [106]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[0.1250, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1250, 0.1429, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1250, 0.1429, 0.1667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1250, 0.1429, 0.1667, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1250, 0.1429, 0.1667, 0.2000, 0.2500, 0.0000, 0.0000, 0.0000],
        [0.1250, 0.1429, 0.1667, 0.2000, 0.2500, 0.3333, 0.0000, 0.0000],
        [0.1250, 0.1429, 0.1667, 0.2000, 0.2500, 0.3333, 0.5000, 0.0000],
        [0.1250, 0.1429, 0.1667, 0.2000, 0.2500, 0.3333, 0.5000, 1.0000]])

In [ ]:
# version with Softmax
trill = torch.tril(torch.ones(T,T)) 
wei = torch.zeros((T,T)) # affinities between tokens
wei = wei.masked_fill(trill == 0, float('-inf')) # future can't communicate with the past
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

# it allows weighted maltiplucation

In [114]:
# self attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channel
x = torch.randn(B,T,C)


In [116]:
# each token emits 2 vectors: query and key
# query - what am I looking for?
# key - what do I contain?
# affinities between tokens in a sequence are calculated as dot product between keys and queries

# a single Head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)


In [120]:
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

In [130]:
k.shape

torch.Size([4, 8, 16])

In [135]:
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

In [136]:
trill = torch.tril(torch.ones(T,T)) 
wei = wei.masked_fill(trill == 0, float('-inf')) # future can't communicate with the past
wei = F.softmax(wei, dim=-1)
out = wei @ x

In [137]:
out.shape

torch.Size([4, 8, 32])

In [139]:
# self attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channel
x = torch.randn(B,T,C)

# a single Head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

trill = torch.tril(torch.ones(T,T)) 
wei = wei.masked_fill(trill == 0, float('-inf')) # future can't communicate with the past
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [140]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [141]:
k.var()

tensor(1.0449)

In [142]:
q.var()

tensor(1.0700)

In [143]:
wei.var()

tensor(1.0918)

In [144]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [145]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])